<a href="https://colab.research.google.com/github/jiwonlee-0218/colab/blob/main/dacon_baseline22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from PIL import Image
from tqdm import tqdm
import os
import cv2
import glob

from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, concatenate, Input, Flatten, Dense
from tensorflow.keras import Model

import warnings
warnings.filterwarnings("ignore")

In [ ]:
TRAIN_IMG_FOLDER = '/content/gdrive/MyDrive/미래연구소/프로젝트/1. open/train_imgs'

TEST_IMG_FOLDER = '/content/gdrive/MyDrive/미래연구소/프로젝트/1. open/test_imgs'

TRAIN_DF = '/content/gdrive/MyDrive/미래연구소/프로젝트/1. open/train_df.csv'
SUB_DF = '/content/gdrive/MyDrive/미래연구소/프로젝트/1. open/sample_submission.csv'

IMAGE_HEIGHT = 540
IMAGE_WIDTH = 960

In [ ]:
!unzip -n '/content/gdrive/MyDrive/미래연구소/프로젝트/1. open.zip' -d '/content/gdrive/MyDrive/미래연구소/프로젝트'

Archive:  /content/gdrive/MyDrive/미래연구소/프로젝트/1. open.zip


In [ ]:
train = pd.read_csv(TRAIN_DF)
submission = pd.read_csv(SUB_DF)

In [ ]:
err=[317, 869, 873, 877, 911, 1559, 1560, 1562, 1566, 1575, 1577, 1578, 1582, 1606, 1607, 1622, 1623, 1624, 1625, 1629, 3968, 4115, 4116, 4117, 4118, 4119, 4120, 4121, 4122, 4123, 4124, 4125, 4126, 4127, 4128, 4129, 4130, 4131, 4132, 4133, 4134, 4135, 4136, 4137, 4138, 4139, 4140, 4141, 4142, 4143, 4144, 4145, 4146, 4147, 4148, 4149, 4150, 4151, 4152, 4153, 4154, 4155, 4156, 4157, 4158, 4159, 4160, 4161, 4162, 4163, 4164, 4165, 4166, 4167, 4168, 4169, 4170, 4171, 4172, 4173, 4174, 4175, 4176, 4177, 4178, 4179, 4180, 4181, 4182, 4183, 4184, 4185, 4186, 4187, 4188, 4189, 4190, 4191, 4192, 4193, 4194]

In [ ]:
train.drop(err, inplace=True) # 라벨링이 잘못된 데이터를 drop

In [ ]:
val_name_set = [] # 이미지 이름 얻기
a = [] # 의미없는 list입니다.

In [ ]:
def val_set(x):
  x1 = x.split('00000')[0]rmsrm
  if a.count(x1) < 2:
    a.append(x1)
    val_name_set.append(x)

In [ ]:
# pandas 에서 apply를 사용해 각 동작마다 두장씩 뽑아서 보기
train['image'].apply(val_set)

In [ ]:
len(val_name_set)

520

In [ ]:
train.describe()

In [ ]:
train.iloc[:,1:49] = train.iloc[:,1:49] / 2 # 1/2 로 리사이즈 할것이기때문
train.head(2)

In [ ]:
train.head(2)

In [ ]:
train.describe()

In [ ]:
#submission 파일 불러오기
submission.head(2)

In [ ]:
train_paths = glob.glob(f'{TRAIN_IMG_FOLDER}/*.jpg')
test_paths = glob.glob(f'{TEST_IMG_FOLDER}/*.jpg')
print(len(train_paths), len(test_paths))

4195 1600


In [ ]:
target_list = list(train.columns)[1:49]

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255
)

train_generator = train_datagen.flow_from_dataframe(
    train,
    directory=TRAIN_IMG_FOLDER,
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    x_col="image",
    y_col=target_list,
    class_mode='raw',
    batch_size=8,
)

test_datagen = ImageDataGenerator(
    rescale=1./255
)

test_generator = test_datagen.flow_from_dataframe(
    submission,
    directory=TEST_IMG_FOLDER,
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    x_col="image",
    y_col=target_list,
    class_mode='raw',
    batch_size=8,
)

Found 4094 validated image filenames.
Found 1600 validated image filenames.


In [ ]:
os.listdir(TRAIN_IMG_FOLDER)

In [ ]:
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.layers import Activation, Convolution2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout, Conv2D,MaxPool2D, ZeroPadding2D

In [ ]:
def create_model():
  model = Sequential()

  model.add(Convolution2D(32, (3,3), padding='same', use_bias=False, input_shape=(IMAGE_HEIGHT,IMAGE_WIDTH,3)))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())

  model.add(Convolution2D(32, (3,3), padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2, 2)))

  model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())

  model.add(Convolution2D(64, (3,3), padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2, 2)))

  model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())

  model.add(Convolution2D(96, (3,3), padding='same', use_bias=False))
  model.add(LeakyReLU(alpha = 0.1))
  model.add(BatchNormalization())
  model.add(MaxPool2D(pool_size=(2, 2)))


  model.add(Flatten())
  model.add(Dense(512,activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(48))

  model.compile(optimizer='adam', 
                loss=tf.keras.losses.MeanSquaredError(),
                metrics=['mae'])
  
  return model

In [ ]:
motion_model = create_model()

In [ ]:
motion_model.fit(train_generator,epochs = 20, verbose=1)

Epoch 1/20
512/512 [==============================] - 141s 276ms/step - loss: 1514.4980 - mae: 29.9722
Epoch 2/20
512/512 [==============================] - 141s 274ms/step - loss: 1494.4208 - mae: 29.9152
Epoch 3/20
512/512 [==============================] - 139s 271ms/step - loss: 1482.4255 - mae: 29.7029
Epoch 4/20
512/512 [==============================] - 139s 270ms/step - loss: 1462.9185 - mae: 29.5078
Epoch 5/20
512/512 [==============================] - 138s 269ms/step - loss: 1475.3839 - mae: 29.7349
Epoch 6/20
512/512 [==============================] - 141s 275ms/step - loss: 1483.9403 - mae: 29.7032
Epoch 7/20
512/512 [==============================] - 142s 278ms/step - loss: 1428.5717 - mae: 29.0354
Epoch 8/20
512/512 [==============================] - 139s 271ms/step - loss: 1405.3450 - mae: 28.7678
Epoch 9/20
512/512 [==============================] - 138s 269ms/step - loss: 1416.5891 - mae: 28.8909
Epoch 10/20
512/512 [==============================] - 138s 269ms/step - 

In [ ]:
!nvidia-smi

Wed Mar 17 15:29:38 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    41W / 250W |  15563MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
pred=motion_model.predict(test_generator)

In [ ]:
submission.iloc[:,1:]=pred * 2

In [ ]:
submission


,image,nose_x,nose_y,left_eye_x,left_eye_y,right_eye_x,right_eye_y,left_ear_x,left_ear_y,right_ear_x,right_ear_y,left_shoulder_x,left_shoulder_y,right_shoulder_x,right_shoulder_y,left_elbow_x,left_elbow_y,right_elbow_x,right_elbow_y,left_wrist_x,left_wrist_y,right_wrist_x,right_wrist_y,left_hip_x,left_hip_y,right_hip_x,right_hip_y,left_knee_x,left_knee_y,right_knee_x,right_knee_y,left_ankle_x,left_ankle_y,right_ankle_x,right_ankle_y,neck_x,neck_y,left_palm_x,left_palm_y,right_palm_x,right_palm_y,spine2(back)_x,spine2(back)_y,spine1(waist)_x,spine1(waist)_y,left_instep_x,left_instep_y,right_instep_x,right_instep_y
0,649-2-4-32-Z148_A-0000001.jpg,846.240540,337.458344,855.323120,328.456696,837.943298,327.966309,865.191101,336.024078,830.527222,334.032745,886.225220,404.676300,789.635681,405.159668,909.689331,450.681305,754.652588,445.583984,902.286255,476.966034,759.480225,473.179962,850.498962,586.484497,796.603699,585.376282,873.319336,688.490601,793.901184,682.351746,894.453552,796.231628,809.635803,791.302612,840.850769,376.188446,898.406677,479.752686,764.380005,479.153290,831.848267,462.719635,823.948853,532.019531,899.296875,816.053406,808.217773,810.536499
1,649-2-4-32-Z148_A-0000003.jpg,949.801819,465.641968,948.032227,457.165161,947.530640,456.118225,945.982544,453.399170,946.276306,449.777008,943.211426,498.342682,951.305725,500.199371,934.455444,570.678772,961.361694,565.966187,942.749146,637.432556,961.139221,635.551147,947.660034,639.896118,959.060364,638.468262,963.457642,746.551880,966.294006,741.677917,986.799133,853.728333,983.233765,853.784485,944.961182,476.057587,946.049438,646.608887,963.670105,648.697632,947.522095,536.211914,949.502869,592.709229,993.121155,873.722107,987.382690,874.039795
2,649-2-4-32-Z148_A-0000005.jpg,1068.264282,503.373810,1063.184448,518.169983,1088.952026,510.820923,1035.364014,550.787109,1093.172363,538.355835,927.566895,548.368652,1073.468872,516.286987,847.039429,544.034546,1101.628540,490.782501,835.318237,495.119141,1085.809570,460.089539,775.643860,537.814331,860.289368,520.204590,717.902710,508.344818,816.901123,486.157990,672.707214,638.403687,761.684143,618.362732,1019.553101,530.915710,836.543579,483.803802,1080.269165,452.698914,939.326416,521.782715,866.788940,520.524170,659.102783,652.562866,752.384888,628.885925
3,649-2-4-32-Z148_A-0000007.jpg,638.003723,495.710602,615.348877,503.013153,629.285034,506.712921,617.814636,536.112427,649.964478,543.539551,670.082214,529.044373,762.297546,563.505371,623.653931,499.118958,784.600708,528.348206,590.618958,480.120361,742.052917,499.703125,943.896118,552.044922,1002.268127,567.858093,1081.997681,518.024048,1161.052979,535.034241,1173.940918,663.038269,1247.863892,696.858765,681.587219,537.335815,592.136597,471.561554,737.944031,492.012024,786.146545,537.650940,881.716431,546.622803,1204.439453,680.230469,1282.127686,716.599365
4,649-2-4-32-Z148_A-0000009.jpg,879.277832,548.047180,884.496704,546.122986,877.983276,542.197510,895.308044,541.107300,879.969116,532.658936,905.073425,557.284363,861.030457,540.622498,895.667053,629.976196,838.053223,595.473206,876.148254,682.026917,818.197876,651.315735,875.541016,616.727356,855.967834,604.584473,885.839600,661.494995,836.884888,643.008240,915.756470,738.068787,860.100708,718.835938,880.890747,541.677185,871.101624,686.554321,814.470032,660.062378,877.081482,559.880920,870.849976,585.692566,913.933777,759.201416,854.355835,737.390015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595,786-3-5-41-Z94_E-0000023.jpg,1119.790649,412.455963,1140.313843,417.211395,1121.531860,410.401428,1152.966309,453.496429,1112.110107,439.439697,1132.720215,508.118317,1003.412170,468.868835,1143.535645,483.520050,922.905701,418.028748,1083.057373,413.079865,918.502930,363.702911,920.821594,631.347656,845.281433,608.122070,858.250366,657.600830,751.523804,

In [ ]:
type(submission)

pandas.core.frame.DataFrame

In [ ]:
submission.to_csv('/content/gdrive/MyDrive/미래연구소/프로젝트/submit.csv', index=False)   # DataFrame.to_csv() -> DataFrame의 데이터를 CSV 파일에 저장